<h2>Capstone Week 4 Project - Data</h2>

#### Import libraries required to complete task

In [1]:
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

In [2]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.1.19-~ --> conda-forge::ca-certificates-

### Download and Explore Dataset

read csv file of Houston Wards and Houston Heights Lat and Lon figures

In [314]:
#read csv file
Hou_wards_df = pd.read_csv("Houston_Wards.csv")
Hou_wards_df

,Neighborhood,Latitude,Longitude
0,First Ward,29.763800,-95.359500
1,Second Ward,29.751100,-95.324000
2,Greater Third Ward,29.727800,-95.349900
3,Fourth Ward,29.757900,-95.380900
4,Greater Fifth Ward,29.776100,-95.326500
5,Old Sixth Ward,29.766339,-95.378272
6,Houston Heights,29.798100,-95.397900


In [4]:
lat = []
lng = []

def getLatLng(row):
    geolocator = Nominatim(user_agent='foursquare')
    print(row[0]+', '+row[1])
    location = geolocator.geocode(row[0]+', '+row[1])
    if location != None:
        lat.append(location.latitude)
        lng.append(location.longitude)
    else:
        lat.append(None)
        lng.append(None)

In [315]:
#import geocoder
from geopy.geocoders import Nominatim

address = 'Houston, Tx'
geolocator = Nominatim(user_agent = "hou_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coords of Houston are {}, {}.'.format(latitude, longitude))

The geographical coords of Houston are 29.7589382, -95.3676974.


Map of Houston with the six Wards and Houston Heights

In [318]:
# create map of New York using latitude and longitude values
map_Hou = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add markers to map
for lat, lng, neighborhood in zip(Hou_wards_df['Latitude'], Hou_wards_df['Longitude'], Hou_wards_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Hou)  

map_Hou

Intialize FourSquare credentials

In [7]:
CLIENT_ID = 'GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ' # your Foursquare ID
CLIENT_SECRET = 'GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ
CLIENT_SECRET:GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH


First Ward

In [8]:
neighborhood_lat_1W = Hou_wards_df.loc[0, 'Latitude']
neighborhood_lon_1W = Hou_wards_df.loc[0, 'Longitude']
neighborhood_nei_1W = Hou_wards_df.loc[0, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_1W, neighborhood_lat_1W, neighborhood_lon_1W))

Lat and Lon values of First Ward are 29.7638, -95.3595.


In [9]:
LIMIT = 100 #radius
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_1W, 
    neighborhood_lon_1W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7638,-95.3595&radius=1000&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60775261caa3be5da1bed3d2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Houston',
  'headerFullLocation': 'Downtown Houston, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 151,
  'suggestedBounds': {'ne': {'lat': 29.772800009000008,
    'lng': -95.34915163063539},
   'sw': {'lat': 29.75479999099999, 'lng': -95.3698483693646}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '541c2faf498e5208ed83297e',
       'name': 'Houston Watch Co.',
       'location': {'address': '913 Franklin St',
        'crossStreet': 'Main St.',
        'lat': 29.763274017872376,
        'lng

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "First Ward"
nearby_venues_1W = nearby_venues
Hou_1W_csv = nearby_venues_1W.to_csv('Hou_1W.csv', index=False)
nearby_venues_1W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Houston Watch Co.,Bar,29.763274,-95.360661,First Ward
1,La Carafe,Wine Bar,29.763041,-95.361904,First Ward
2,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812,First Ward
3,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763,First Ward
4,La Fisheria,Mexican Restaurant,29.763684,-95.362186,First Ward
5,Public Services Wine & Whiskey,Wine Bar,29.763509,-95.361135,First Ward
6,Captain Foxheart's Bad News Bar & Spirits Lodge,Cocktail Bar,29.761987,-95.361030,First Ward
7,Market Square Park,Park,29.762556,-95.362137,First Ward
8,Boomtown Coffee,Coffee Shop,29.762283,-95.360976,First Ward
9,Conservatory,Beer Garden,29.760427,-95.361570,First Ward


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Second Ward Data

In [14]:
neighborhood_lat_2W = Hou_wards_df.loc[1, 'Latitude']
neighborhood_lon_2W = Hou_wards_df.loc[1, 'Longitude']
neighborhood_nei_2W = Hou_wards_df.loc[1, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_2W, neighborhood_lat_2W, neighborhood_lon_2W))

Lat and Lon values of Second Ward are 29.7511, -95.324.


In [163]:
LIMIT = 100 #radius
radius = 2500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_2W, 
    neighborhood_lon_2W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7511,-95.324&radius=2500&limit=100'

In [164]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607758907ed53c21e51f9f18'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Houston',
  'headerFullLocation': 'Houston',
  'headerLocationGranularity': 'city',
  'totalResults': 72,
  'suggestedBounds': {'ne': {'lat': 29.773600022500023,
    'lng': -95.29813235488766},
   'sw': {'lat': 29.72859997749998, 'lng': -95.34986764511234}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50afbb77e4b0211a31a22d82',
       'name': 'Texas Mattress Makers',
       'location': {'address': '4619 Navigation Blvd',
        'crossStreet': 'Lockwood Dr',
        'lat': 29.752096010409602,
        'lng': -95.3237248121643,
 

In [113]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [114]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Second Ward"
nearby_venues_2W = nearby_venues
Hou_2W_csv = nearby_venues_2W.to_csv('Hou_2W.csv', index=False)
nearby_venues_2W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Texas Mattress Makers,Mattress Store,29.752096,-95.323725,Second Ward
1,"""La Silla"" Pollos asados al carbon",Mexican Restaurant,29.748268,-95.327063,Second Ward
2,The White Swan,Bar,29.752043,-95.326671,Second Ward
3,Voodoo Queen,Dive Bar,29.747480,-95.337828,Second Ward
4,Refresqueria Tampico,Snack Place,29.743250,-95.326179,Second Ward
5,Champ Burger,American Restaurant,29.749796,-95.340350,Second Ward
6,El Tiempo Cantina,Mexican Restaurant,29.756222,-95.341484,Second Ward
7,Moon Tower Inn,Beer Garden,29.754298,-95.341045,Second Ward
8,Lil’ Danny Speedo’s Go Fly a Kite Lounge,Dive Bar,29.738970,-95.326802,Second Ward
9,The Original Ninfa's on Navigation,Mexican Restaurant,29.756859,-95.342606,Second Ward


Third Ward Data

In [115]:
neighborhood_lat_3W = Hou_wards_df.loc[2, 'Latitude']
neighborhood_lon_3W = Hou_wards_df.loc[2, 'Longitude']
neighborhood_nei_3W = Hou_wards_df.loc[2, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_3W, neighborhood_lat_3W, neighborhood_lon_3W))

Lat and Lon values of Greater Third Ward are 29.7278, -95.3499.


In [169]:
# type your answer here
#manhattan_data.loc[0:100, 'Neighborhood']

LIMIT = 100 #radius
radius = 2500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_3W, 
    neighborhood_lon_3W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7278,-95.3499&radius=2500&limit=100'

In [170]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607758bfa749fd6cccdc4ae5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Houston',
  'headerFullLocation': 'Houston',
  'headerLocationGranularity': 'city',
  'totalResults': 118,
  'suggestedBounds': {'ne': {'lat': 29.75030002250002,
    'lng': -95.32403836394184},
   'sw': {'lat': 29.705299977499976, 'lng': -95.37576163605817}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ab03208f964a520626620e3',
       'name': 'Moores Opera House',
       'location': {'address': 'Elgin',
        'lat': 29.725691661022903,
        'lng': -95.34439828714129,
        'labeledLatLngs': [{'label': 'display',
       

In [118]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [119]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Third Ward"
nearby_venues_3W = nearby_venues
Hou_3W_csv = nearby_venues_3W.to_csv('Hou_3W.csv', index=False)
nearby_venues_3W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Moores Opera House,Opera House,29.725692,-95.344398,Third Ward
1,The Library Coffee & Wine House,Coffee Shop,29.732206,-95.348890,Third Ward
2,Fertitta Center,Basketball Stadium,29.724703,-95.347155,Third Ward
3,Chick-fil-A,Food Service,29.721600,-95.340594,Third Ward
4,Project Row Houses,Art Gallery,29.731756,-95.364882,Third Ward
5,Frenchy's Chicken,Fried Chicken Joint,29.718851,-95.352467,Third Ward
6,Barebowls Kitchen,Food Truck,29.720076,-95.341536,Third Ward
7,The Nook Cafe,Coffee Shop,29.722773,-95.337208,Third Ward
8,Chick-fil-A,Food Service,29.720994,-95.361085,Third Ward
9,Mandola's Deli,Italian Restaurant,29.737454,-95.339587,Third Ward


Fourth Ward Data

In [120]:
neighborhood_lat_4W = Hou_wards_df.loc[3, 'Latitude']
neighborhood_lon_4W = Hou_wards_df.loc[3, 'Longitude']
neighborhood_nei_4W = Hou_wards_df.loc[3, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_4W, neighborhood_lat_4W, neighborhood_lon_4W))

Lat and Lon values of Fourth Ward are 29.7579, -95.3809.


In [121]:
LIMIT = 100 #radius
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_4W, 
    neighborhood_lon_4W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7579,-95.3809&radius=1000&limit=100'

In [122]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6077586c69a5f62da77c6cf0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Fourth Ward',
  'headerFullLocation': 'Fourth Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 29.766900009000008,
    'lng': -95.3705522399357},
   'sw': {'lat': 29.74889999099999, 'lng': -95.39124776006429}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bff00bcca1920a16f32ec81',
       'name': 'Eleanor Tinsley Park',
       'location': {'address': '500 Allen Parkway',
        'lat': 29.761439843452138,
        'lng': -95.37927095656195,
        'labeledLa

In [123]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [124]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Fourth Ward"
nearby_venues_4W = nearby_venues
Hou_4W_csv = nearby_venues_4W.to_csv('Hou_4W.csv', index=False)
nearby_venues_4W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Eleanor Tinsley Park,Park,29.761440,-95.379271,Fourth Ward
1,Oporto Fooding House & Wine,Portuguese Restaurant,29.753179,-95.380243,Fourth Ward
2,Buffalo Bayou Walk,Trail,29.762177,-95.375844,Fourth Ward
3,Paper Street Crossfit,Gym,29.757435,-95.385846,Fourth Ward
4,Buffalo Bayou Cistern,Public Art,29.762214,-95.375873,Fourth Ward
5,Lucio's BYOB,Bar,29.758326,-95.385591,Fourth Ward
6,Michael Saldana Salon,Cosmetics Shop,29.752019,-95.381208,Fourth Ward
7,Houston Police Officer Memorial,Sculpture Garden,29.762893,-95.383077,Fourth Ward
8,The Fish Restaurant & Sushi Bar,Sushi Restaurant,29.752249,-95.376820,Fourth Ward
9,Front Porch Pub,Pub,29.752643,-95.377453,Fourth Ward


Fifth Ward Data

In [125]:
neighborhood_lat_5W = Hou_wards_df.loc[4, 'Latitude']
neighborhood_lon_5W = Hou_wards_df.loc[4, 'Longitude']
neighborhood_nei_5W = Hou_wards_df.loc[4, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_5W, neighborhood_lat_5W, neighborhood_lon_5W))

Lat and Lon values of Greater Fifth Ward are 29.7761, -95.3265.


In [173]:
LIMIT = 100 #radius
radius = 3000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_5W, 
    neighborhood_lon_5W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7761,-95.3265&radius=3000&limit=100'

In [175]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6077595908ffa619cdbf88b0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Houston',
  'headerFullLocation': 'Houston',
  'headerLocationGranularity': 'city',
  'totalResults': 70,
  'suggestedBounds': {'ne': {'lat': 29.803100027000028,
    'lng': -95.29545107944637},
   'sw': {'lat': 29.74909997299997, 'lng': -95.35754892055363}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c3e57706faac9b6a7230e76',
       'name': 'Frenchys Chicken',
       'location': {'address': '4403 East Fwy',
        'lat': 29.773792967574607,
        'lng': -95.32607797848243,
        'labeledLatLngs': [{'label': 'display',
  

In [176]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [177]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Fifth Ward"
nearby_venues_5W = nearby_venues
Hou_5W_csv = nearby_venues_5W.to_csv('Hou_5W.csv', index=False)
nearby_venues_5W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Frenchys Chicken,Fried Chicken Joint,29.773793,-95.326078,Fifth Ward
1,Silo,Music Venue,29.767566,-95.324330,Fifth Ward
2,The New Potato,Pub,29.765021,-95.334025,Fifth Ward
3,Saint Arnold Beer Garden,Beer Garden,29.770962,-95.347665,Fifth Ward
4,Saint Arnold Brewing Company,Brewery,29.771175,-95.348588,Fifth Ward
5,J & J Lounge,American Restaurant,29.781979,-95.336325,Fifth Ward
6,El Tiempo Cantina,Mexican Restaurant,29.756222,-95.341484,Fifth Ward
7,The Original Ninfa's on Navigation,Mexican Restaurant,29.756859,-95.342606,Fifth Ward
8,Texas Mattress Makers,Mattress Store,29.752096,-95.323725,Fifth Ward
9,Moon Tower Inn,Beer Garden,29.754298,-95.341045,Fifth Ward


Sixth Ward

In [178]:
neighborhood_lat_6W = Hou_wards_df.loc[5, 'Latitude']
neighborhood_lon_6W = Hou_wards_df.loc[5, 'Longitude']
neighborhood_nei_6W = Hou_wards_df.loc[5, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_6W, neighborhood_lat_6W, neighborhood_lon_6W))

Lat and Lon values of Old Sixth Ward are 29.7663391, -95.3782719.


In [179]:
LIMIT = 100 #radius
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_6W, 
    neighborhood_lon_6W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7663391,-95.3782719&radius=1000&limit=100'

In [180]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6077595a7f6c785fd82f78ae'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Sixth Ward',
  'headerFullLocation': 'Sixth Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 74,
  'suggestedBounds': {'ne': {'lat': 29.775339109000008,
    'lng': -95.36792326836353},
   'sw': {'lat': 29.75733909099999, 'lng': -95.38862053163648}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5033e2e3e4b0ee26f2d6aa57',
       'name': 'Julep',
       'location': {'address': '1919 Washington Ave.',
        'crossStreet': 'White St.',
        'lat': 29.767723245110254,
        'lng': -95.37808149627

In [181]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [182]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Sixth Ward"
nearby_venues_6W = nearby_venues
Hou_6W_csv = nearby_venues_6W.to_csv('Hou_6W.csv', index=False)
nearby_venues_6W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Julep,Cocktail Bar,29.767723,-95.378081,Sixth Ward
1,Tacodeli,Taco Place,29.767928,-95.377333,Sixth Ward
2,Gus's World Famous Fried Chicken,Fried Chicken Joint,29.767542,-95.376473,Sixth Ward
3,Catalina Coffee,Coffee Shop,29.767968,-95.380662,Sixth Ward
4,Platypus Brewing,Brewery,29.768071,-95.377502,Sixth Ward
5,B & B Butchers,Steakhouse,29.767933,-95.376494,Sixth Ward
6,Taqueria Barba,Food Truck,29.767813,-95.380799,Sixth Ward
7,Washington Gym,Gym,29.767661,-95.378145,Sixth Ward
8,Henderson Heights,Gastropub,29.767525,-95.379302,Sixth Ward
9,Buffalo Bayou Walk,Trail,29.762177,-95.375844,Sixth Ward


Houston Heights

In [183]:
neighborhood_lat_HH = Hou_wards_df.loc[6, 'Latitude']
neighborhood_lon_HH = Hou_wards_df.loc[6, 'Longitude']
neighborhood_nei_HH = Hou_wards_df.loc[6, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_HH, neighborhood_lat_HH, neighborhood_lon_HH))

Lat and Lon values of Houston Heights are 29.7981, -95.3979.


In [184]:
LIMIT = 100 #radius
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_HH, 
    neighborhood_lon_HH, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7981,-95.3979&radius=1000&limit=100'

In [185]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6077595c8353ca1e4bc87dc7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Greater Heights',
  'headerFullLocation': 'Greater Heights, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 29.80710000900001,
    'lng': -95.38754808483564},
   'sw': {'lat': 29.789099990999993, 'lng': -95.40825191516437}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55f4747d498e264103800fc3',
       'name': 'The Wilde Collection',
       'location': {'address': '1446 Yale St',
        'crossStreet': '15th',
        'lat': 29.797919585722735,
        'lng': -95

In [186]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [187]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Hou Heights"
nearby_venues_HH = nearby_venues
Hou_HH_csv = nearby_venues_HH.to_csv('Hou_HH.csv', index=False)
nearby_venues_HH

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,The Wilde Collection,Gift Shop,29.797920,-95.398890,Hou Heights
1,Cosa Bella Salon & Day Spa,Salon / Barbershop,29.799587,-95.399416,Hou Heights
2,La Paz Day Spa,Spa,29.796223,-95.398476,Hou Heights
3,Heights Running Trail,Trail,29.802266,-95.398002,Hou Heights
4,Boomtown Coffee,Coffee Shop,29.802849,-95.400855,Hou Heights
5,Down House,New American Restaurant,29.802214,-95.399355,Hou Heights
6,Opera In The Heights,Opera House,29.801161,-95.398474,Hou Heights
7,Marmion Park,Park,29.802301,-95.397266,Hou Heights
8,Manready Mercantile,Men's Store,29.803043,-95.402263,Hou Heights
9,Torchy's Tacos,Taco Place,29.802660,-95.403417,Hou Heights


Combined Wards and Hou Heights DF

In [188]:
csv_file_list = ["Hou_1W.csv","Hou_2W.csv","Hou_3W.csv","Hou_4W.csv","Hou_5W.csv","Hou_6W.csv","Hou_HH.csv"]
list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))
merged_df = pd.concat(list_of_dataframes)

In [333]:
#Number of venues per Ward
merged_df.groupby(['Ward'], as_index=False)['categories'].count().sort_values('categories')
#merged_df.value_counts()

,Ward,categories
0,Fifth Ward,70
5,Sixth Ward,74
2,Fourth Ward,75
3,Hou Heights,90
4,Second Ward,90
1,First Ward,100
6,Third Ward,100


In [329]:
#Displays the venue category frequency per Ward
#merged_df.groupby(['categories','Ward')['categories'].count().sort_values()

In [190]:
print(merged_df.shape[0],"venues within the Houston Wards and Hou Heights")
merged_df.reset_index(inplace=True)

599 venues within the Houston Wards and Hou Heights


In [191]:
#merged_df.groupby('Ward').count()
Hou_merged = merged_df[merged_df["categories"] != "Intersection"].reset_index(drop=True)
Hou_merged_1 = Hou_merged[Hou_merged["categories"] != "Park"].reset_index(drop=True)
Hou_merged_2 = Hou_merged_1[Hou_merged_1["categories"] != "Trail"].reset_index(drop=True)
#df_new = df.drop(df[(df['col_1'] == 1.0) & (df['col_2'] == 0.0)].index) #clearner method to consider

#Hou_merged_2.reset_index(drop=True) #display full table
Hou_WardHH_DF = Hou_merged_2.drop(['index'], axis=1)
Hou_WardHH_DF[:5]

,name,categories,lat,lng,Ward
0,Houston Watch Co.,Bar,29.763274,-95.360661,First Ward
1,La Carafe,Wine Bar,29.763041,-95.361904,First Ward
2,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812,First Ward
3,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763,First Ward
4,La Fisheria,Mexican Restaurant,29.763684,-95.362186,First Ward


Analyze each Ward

In [192]:
# move neighborhood column to the first column
Hou_WardHH_DF_onehot = pd.get_dummies(Hou_WardHH_DF[['categories']], prefix="", prefix_sep="")

Hou_WardHH_DF_onehot['Ward'] = Hou_WardHH_DF['Ward']

fixed_columns = [Hou_WardHH_DF_onehot.columns[-1]] + list(Hou_WardHH_DF_onehot.columns[:-1])
Hou_WardHH_DF_onehot = Hou_WardHH_DF_onehot[fixed_columns]

Hou_WardHH_DF_onehot[:5]

,Ward,Airport Service,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Cajun / Creole Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dry Cleaner,Escape Room,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Latin American Restaurant,Lawyer,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Public Art,Record Shop,Restaurant,River,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse,Wine Bar,Wings Joint,Women's Store
0,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [193]:
Hou_WardHH_DF_onehot.shape[0]

569

In [194]:
Hou_WardHH_DF_grouped = Hou_WardHH_DF_onehot.groupby('Ward').mean().reset_index()
Hou_WardHH_DF_grouped

,Ward,Airport Service,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Cajun / Creole Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dry Cleaner,Escape Room,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Latin American Restaurant,Lawyer,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Public Art,Record Shop,Restaurant,River,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse,Wine Bar,Wings Joint,Women's Store
0,Fifth Ward,0.000000,0.014706,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044118,0.000000,0.000000,0.000000,0.000000,0.000000,0.088235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.014706,0.000000,0.088235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.014706,0.014706,0.000000,0.000000,0.132353,0.014706,0.000000,0.000000,0.000000,0.014706,0.029412,0.000000,0.000000,0.014706,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.029412,0.014706,0.029412,0.000000,0.014706,0.000000,0.014706,0.000000,0.000000,0.029412,0.000000,0.000000,0.044118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.014706,0.000000,0.000000,0.000000
1,First Ward,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.091837,0.000000,0.030612,0.00000

In [195]:
num_top_venues = 5

for hood in Hou_WardHH_DF_grouped['Ward']:
    print("----"+hood+"----")
    temp = Hou_WardHH_DF_grouped[Hou_WardHH_DF_grouped['Ward'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Fifth Ward----
                  venue  freq
0    Mexican Restaurant  0.13
1  Fast Food Restaurant  0.09
2           Gas Station  0.09
3   Fried Chicken Joint  0.06
4        Sandwich Place  0.04


----First Ward----
                venue  freq
0                 Bar  0.09
1               Hotel  0.08
2        Cocktail Bar  0.07
3         Coffee Shop  0.05
4  Mexican Restaurant  0.04


----Fourth Ward----
                 venue  freq
0                  Bar  0.06
1          Coffee Shop  0.05
2  American Restaurant  0.05
3       Sandwich Place  0.05
4                  Pub  0.03


----Hou Heights----
          venue  freq
0   Coffee Shop  0.05
1   Flower Shop  0.03
2        Bakery  0.03
3  Burger Joint  0.02
4      Pharmacy  0.02


----Second Ward----
                  venue  freq
0    Mexican Restaurant  0.11
1  Fast Food Restaurant  0.06
2                   Bar  0.05
3              Pharmacy  0.05
4     Convenience Store  0.05


----Sixth Ward----
                  venue  freq
0  Fast F

In [253]:
#Hou_WardHH_DF = Hou_WardHH_DF[['Ward', 'lat', 'lng']]
#Hou_WardHH_DF[:5]

Hou_wards_df.columns = ['Ward', 'lat', 'lng']
Hou_wards_df

,Ward,lat,lng
0,First Ward,29.763800,-95.359500
1,Second Ward,29.751100,-95.324000
2,Greater Third Ward,29.727800,-95.349900
3,Fourth Ward,29.757900,-95.380900
4,Greater Fifth Ward,29.776100,-95.326500
5,Old Sixth Ward,29.766339,-95.378272
6,Houston Heights,29.798100,-95.397900


In [282]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

In [283]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Ward']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Ward'] = Hou_WardHH_DF_grouped['Ward']

for ind in np.arange(Hou_WardHH_DF_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Hou_WardHH_DF_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fifth Ward,Mexican Restaurant,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Sandwich Place,Discount Store,Restaurant,Bank,Pizza Place,Beer Garden
1,First Ward,Bar,Hotel,Cocktail Bar,Coffee Shop,Burger Joint,Italian Restaurant,Mexican Restaurant,Pizza Place,Baseball Stadium,Performing Arts Venue
2,Fourth Ward,Bar,Coffee Shop,American Restaurant,Sandwich Place,Hotel,Gym,Restaurant,Pizza Place,Bank,Pub
3,Hou Heights,Coffee Shop,Bakery,Flower Shop,Furniture / Home Store,Sushi Restaurant,Gym / Fitness Center,Sandwich Place,Gift Shop,Restaurant,Juice Bar
4,Second Ward,Mexican Restaurant,Fast Food Restaurant,Bar,Pharmacy,Convenience Store,Music Venue,Dive Bar,Bakery,Snack Place,Bank
5,Sixth Ward,Brewery,Gym,Art Gallery,Fast Food Restaurant,Food Truck,Athletics & Sports,Burger Joint,Clothing Store,Sculpture Garden,Seafood Restaurant
6,Third Ward,Bar,Coffee Shop,Vietnamese Restaurant,Brewery,Food Service,Bakery,American Restaurant,Dive Bar,Burger Joint,Beer Garden


In [290]:
#neighborhoods_ven_sorted_csv = neighborhoods_venues_sorted.to_csv("neighborhoods_venues_sorted.csv")

Hou_venue_merged = pd.read_csv("neighborhoods_venues_sorted_Hou_Wards.csv")
Hou_venue_merged
#Tron_Combined_csv = Tron_Combined.to_csv('Tron_Combined.csv', index=False)

,Ward,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fifth Ward,29.776100,-95.326500,0,Mexican Restaurant,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Sandwich Place,Discount Store,Restaurant,Bank,Pizza Place,Beer Garden
1,First Ward,29.763800,-95.359500,1,Bar,Hotel,Cocktail Bar,Coffee Shop,Burger Joint,Italian Restaurant,Mexican Restaurant,Pizza Place,Baseball Stadium,Performing Arts Venue
2,Fourth Ward,29.757900,-95.380900,3,Bar,Coffee Shop,American Restaurant,Sandwich Place,Hotel,Gym,Restaurant,Pizza Place,Bank,Pub
3,Hou Heights,29.798100,-95.397900,3,Coffee Shop,Bakery,Flower Shop,Furniture / Home Store,Sushi Restaurant,Gym / Fitness Center,Sandwich Place,Gift Shop,Restaurant,Juice Bar
4,Second Ward,29.751100,-95.324000,0,Mexican Restaurant,Fast Food Restaurant,Bar,Pharmacy,Convenience Store,Music Venue,Dive Bar,Bakery,Snack Place,Bank
5,Sixth Ward,29.766339,-95.378272,2,Brewery,Gym,Art Gallery,Fast Food Restaurant,Food Truck,Athletics & Sports,Burger Joint,Clothing Store,Sculpture Garden,Seafood Restaurant
6,Third Ward,29.727800,-95.349900,1,Bar,Coffee Shop,Vietnamese Restaurant,Brewery,Food Service,Bakery,American Restaurant,Dive Bar,Burger Joint,Beer Garden


In [285]:
# set number of clusters
kclusters = 4

Hou_WardHH_DF_grouped_clustering = Hou_WardHH_DF_grouped.drop('Ward', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Hou_WardHH_DF_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 3, 3, 0, 2, 1], dtype=int32)

In [286]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [287]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#Hou_wards_df_merged = Hou_WardHH_DF

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
#Hou_wards_df_merged = Hou_wards_df_merged.join(neighborhoods_venues_sorted.set_index('Ward'), on='Ward')

#Hou_wards_df_merged # check the last columns!

#Does not capture all of the venues
#Hou_wards_df = Hou_wards_df.join(neighborhoods_venues_sorted.set_index('Ward'), on='Ward')
#Hou_wards_df

In [334]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
#for lat, lon, poi, cluster in zip(Hou_wards_df_merged['lat'], Hou_wards_df_merged['lng'], Hou_wards_df_merged['Ward'], Hou_wards_df_merged['Cluster Labels']):
for lat, lon, poi, cluster in zip(Hou_venue_merged['Latitude'], Hou_venue_merged['Longitude'], Hou_venue_merged['Ward'], Hou_venue_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

Cluster 1

In [292]:
Hou_venue_merged.loc[Hou_venue_merged['Cluster Labels'] == 0, Hou_venue_merged.columns[[0] + list(range(4, Hou_venue_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fifth Ward,Mexican Restaurant,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Sandwich Place,Discount Store,Restaurant,Bank,Pizza Place,Beer Garden
4,Second Ward,Mexican Restaurant,Fast Food Restaurant,Bar,Pharmacy,Convenience Store,Music Venue,Dive Bar,Bakery,Snack Place,Bank


Cluster 2

In [293]:
Hou_venue_merged.loc[Hou_venue_merged['Cluster Labels'] == 1, Hou_venue_merged.columns[[0] + list(range(4, Hou_venue_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,First Ward,Bar,Hotel,Cocktail Bar,Coffee Shop,Burger Joint,Italian Restaurant,Mexican Restaurant,Pizza Place,Baseball Stadium,Performing Arts Venue
6,Third Ward,Bar,Coffee Shop,Vietnamese Restaurant,Brewery,Food Service,Bakery,American Restaurant,Dive Bar,Burger Joint,Beer Garden


Cluster 3

In [294]:
Hou_venue_merged.loc[Hou_venue_merged['Cluster Labels'] == 2, Hou_venue_merged.columns[[0] + list(range(4, Hou_venue_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Sixth Ward,Brewery,Gym,Art Gallery,Fast Food Restaurant,Food Truck,Athletics & Sports,Burger Joint,Clothing Store,Sculpture Garden,Seafood Restaurant


Cluster 4

In [295]:
Hou_venue_merged.loc[Hou_venue_merged['Cluster Labels'] == 3, Hou_venue_merged.columns[[0] + list(range(4, Hou_venue_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Fourth Ward,Bar,Coffee Shop,American Restaurant,Sandwich Place,Hotel,Gym,Restaurant,Pizza Place,Bank,Pub
3,Hou Heights,Coffee Shop,Bakery,Flower Shop,Furniture / Home Store,Sushi Restaurant,Gym / Fitness Center,Sandwich Place,Gift Shop,Restaurant,Juice Bar
